In [79]:
%load_ext autoreload
%autoreload 2
%pylab inline

import pyspiel
import open_spiel.python.examples.ubc_utils
from open_spiel.python.visualizations import ubc_treeviz
from open_spiel.python.examples.ubc_utils import *
from absl import logging
logging.set_verbosity(logging.INFO)
from open_spiel.python.examples.ubc_nfsp_example import NFSPPolicies, policy_from_checkpoint
from open_spiel.python.examples.ubc_br import checkpoint_sub_agent_i

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


In [80]:
### Load game
game_name = 'small_game_2'
game = smart_load_sequential_game('clock_auction', game_parameters=dict(filename=f'{game_name}.json'))


In [82]:
EXPERIMENT_DIR = '/shared/outputs/jan14_small/mlp'
# CHECKPOINT_NAME = 'checkpoint_latest'
CHECKPOINT_NAME = 'checkpoint_1500000'

In [83]:
# game.new_initial_state().information_state_tensor(0)

In [84]:
policy = policy_from_checkpoint(EXPERIMENT_DIR, CHECKPOINT_NAME).nfsp_policies

INFO:absl:Game loaded
INFO:absl:Using game instance: turn_based_simultaneous_game
INFO:absl:Game has a state size of 4009, 4 distinct actions, and 2 players
INFO:absl:Game has 2 products


In [85]:
# To load a BR
# policy = checkpoint_sub_agent_i(EXPERIMENT_DIR, 'checkpoint_3000000', 'checkpoint_3000000_br_0')

In [86]:
# solver = pyspiel.CFRSolver(game)
# for i in range(1000):
#     solver.evaluate_and_update_policy()
# policy = solver.average_policy()

In [87]:
def make_policy_decorators(policy):
    def edge_weight_by_policy_decorator(parent_state, unused_child, action, **kwargs):
        attrs = ubc_treeviz.default_edge_decorator(parent_state, unused_child, action)  # get default attributes
        if parent_state.current_player() < 0:
            return attrs
        
        if '@ $0' in attrs['label']:
            attrs['label'] = 'Drop Out'
        
        action_prob = policy.action_probabilities(parent_state)[action]
        attrs['arrowsize'] = action_prob * attrs['arrowsize']
        attrs['penwidth'] = action_prob
        attrs['label'] = f'[{action_prob:.2f}] {attrs["label"]}'
#         print(attrs, parent_state.current_player())
        return attrs

    def node_weight_by_policy_decorator(state, **kwargs):
        attrs = ubc_treeviz.default_node_decorator(state)
        if state.is_terminal():
            return_list = eval(attrs['label'])
            attrs['label'] = ', '.join([f'{x:.2f}' for x in return_list])
        
        # TODO: If terminal, report allocation
        # {'label': 'Current player: 0\np0v125, 125b150\n', 'fontsize': 8, 'width': 0.25, 'height': 0.25, 'margin': 0.01, 'shape': 'square', 'color': 'blue'}
        #         print(attrs)
        return attrs

    return node_weight_by_policy_decorator, edge_weight_by_policy_decorator

node_policy_decorator, edge_policy_decorator = make_policy_decorators(policy)

#   player = parent.current_player()
#   attrs = {
#       "label": " " + parent.action_to_string(player, action),
#       "fontsize": _FONTSIZE,
#       "arrowsize": _ARROWSIZE
#   }
#   attrs["color"] = _PLAYER_COLORS.get(player, "black")
#   return attrs


In [91]:
gametree = ubc_treeviz.GameTree(
    game,
    node_decorator=node_policy_decorator,
    edge_decorator=edge_policy_decorator,
    group_infosets=True,
    group_terminal=False,
    group_pubsets=False, 
    target_pubset='*',
    depth_limit=10,
#     state_prob_limit=0.01,
    action_prob_limit=0.01, 
    policy=policy
)

# outfile = f'./{game_name}.pdf'
outfile= f'./{game_name}_rnn.pdf'

# logging.info("Game tree:\n%s", gametree.to_string())

gametree.draw(outfile, prog='dot')
logging.info("Game tree saved to file: %s", outfile)


INFO:absl:Building tree...
INFO:absl:Built tree!
INFO:absl:Game tree saved to file: ./small_game_2_rnn.pdf


In [62]:
# TODO: Given a STRATEGY, colour the lines according to thickness?

In [34]:
env_and_model = policy_from_checkpoint(EXPERIMENT_DIR, CHECKPOINT_NAME)
game, policy, env, trained_agents, game_config = env_and_model.game, env_and_model.nfsp_policies, env_and_model.env, env_and_model.agents, env_and_model.game_config


INFO:absl:Game loaded
INFO:absl:Using game instance: turn_based_simultaneous_game
INFO:absl:Game has a state size of 26, 12 distinct actions, and 2 players
INFO:absl:Game has 3 products


In [43]:
check_on_q_values(trained_agents[0]._rl_agent, game)

{'Bid for 0,0,0 licenses @ $0 with activity 0': tensor(nan),
 'Bid for 0,0,1 licenses @ $25 with activity 25': tensor(nan),
 'Bid for 0,1,0 licenses @ $50 with activity 50': tensor(nan),
 'Bid for 0,1,1 licenses @ $75 with activity 75': tensor(nan),
 'Bid for 1,0,0 licenses @ $75 with activity 75': tensor(nan),
 'Bid for 1,0,1 licenses @ $100 with activity 100': tensor(nan),
 'Bid for 1,1,0 licenses @ $125 with activity 125': tensor(nan),
 'Bid for 1,1,1 licenses @ $150 with activity 150': tensor(nan),
 'Bid for 2,0,0 licenses @ $150 with activity 150': tensor(nan),
 'Bid for 2,0,1 licenses @ $175 with activity 175': tensor(nan),
 'Bid for 2,1,0 licenses @ $200 with activity 200': tensor(nan),
 'Bid for 2,1,1 licenses @ $225 with activity 225': tensor(nan)}

In [74]:
# import glob
# for script in glob.glob('/shared/outputs/jan10_handcrafted_7/mlp-low/eval*straight*'):
#     !sbatch {script}

Submitted batch job 4118
Submitted batch job 4119
Submitted batch job 4120
Submitted batch job 4121
Submitted batch job 4122
Submitted batch job 4123
Submitted batch job 4124
Submitted batch job 4125
Submitted batch job 4126
Submitted batch job 4127
Submitted batch job 4128
Submitted batch job 4129
Submitted batch job 4130
Submitted batch job 4131
Submitted batch job 4132
Submitted batch job 4133
Submitted batch job 4134
Submitted batch job 4135
Submitted batch job 4136
Submitted batch job 4137
